In [1]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

2025-12-15 19:21:24.751043: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 19:21:24.752524: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 19:21:24.781242: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 19:21:24.781772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 19:21:25.618210: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [4]:
wisdm_dataset_path = f'{base_path}/data/WISDM'
wisdm_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/WISDM'

In [5]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [6]:
import src.load_wisdm_dataset as wisdm

In [7]:
Xtrain, ytrain, xtest, ytest, labels = wisdm.load_wisdm_dir(wisdm_dataset_path)

In [8]:
Xtrain.shape, ytrain.shape, xtest.shape, ytest.shape, labels,len(labels)

((13639, 128, 6),
 (13639,),
 (3519, 128, 6),
 (3519,),
 ['downstairs', 'jogging', 'sitting', 'standing', 'upstairs', 'walking'],
 6)

In [9]:
import src.get_models as models

In [10]:
input_shape = Xtrain.shape[1:]
num_classes = len(labels)
input_shape, num_classes

((128, 6), 6)

In [11]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [13]:
base_model = keras.models.load_model(f'{base_path}/models/cnn_lstm_wisdm_model.keras')

2025-12-15 19:22:38.921289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 19:22:38.964581: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-12-15 19:22:39.136066: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [14]:
base_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 128, 64)           1984      
                                                                 
 batch_normalization_3 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 re_lu_2 (ReLU)              (None, 128, 64)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 128, 128)          24704     
                                                                 
 batch_normalization_4 (Batc  (None, 128, 128)         512       
 hNormalization)                                           

In [15]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 19:22:44.884516: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:22:44.885520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:22:44.886184: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

110/110 [==============================] - 3s 27ms/step - loss: 0.8923 - accuracy: 0.8346
Baseline test accuracy: 0.8346121311187744


In [16]:
q_aware_model = models.get_qat_model_dense_only(input_shape, num_classes)

2025-12-15 19:22:50.019207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:22:50.020397: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:22:50.021141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:22:51.086065: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:22:51.086945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:22:51.087551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [17]:
q_aware_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [18]:
q_aware_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 128, 64)           1984      
                                                                 
 batch_normalization (BatchN  (None, 128, 64)          256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 128)          24704     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128)         512       
 hNormalization)                                             

In [19]:
lr_decay_callback = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                      patience=8, min_lr=1e-6, verbose=1)
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
q_aware_model.fit(Xtrain, ytrain, epochs=100, batch_size=64,
                          validation_split=0.2, verbose=1, callbacks=[early_stop, lr_decay_callback])

Epoch 1/100


2025-12-15 19:24:06.004339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:24:06.005412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:24:06.006182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

171/171 [==============================] - ETA: 0s - loss: 0.5294 - accuracy: 0.8224

2025-12-15 19:24:23.777084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:24:23.777971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:24:23.778584: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

171/171 [==============================] - 20s 107ms/step - loss: 0.5294 - accuracy: 0.8224 - val_loss: 0.8591 - val_accuracy: 0.6943 - lr: 0.0010
Epoch 2/100
171/171 [==============================] - 18s 104ms/step - loss: 0.2985 - accuracy: 0.9011 - val_loss: 1.0424 - val_accuracy: 0.6906 - lr: 0.0010
Epoch 3/100
171/171 [==============================] - 18s 104ms/step - loss: 0.2097 - accuracy: 0.9288 - val_loss: 1.2471 - val_accuracy: 0.6994 - lr: 0.0010
Epoch 4/100
171/171 [==============================] - 18s 104ms/step - loss: 0.1605 - accuracy: 0.9485 - val_loss: 1.1246 - val_accuracy: 0.7089 - lr: 0.0010
Epoch 5/100
171/171 [==============================] - 18s 105ms/step - loss: 0.1263 - accuracy: 0.9605 - val_loss: 1.0940 - val_accuracy: 0.7823 - lr: 0.0010
Epoch 6/100
171/171 [==============================] - 18s 104ms/step - loss: 0.1333 - accuracy: 0.9581 - val_loss: 0.8544 - val_accuracy: 0.8054 - lr: 0.0010
Epoch 7/100
171/171 [==============================] - 18s

In [20]:
q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
   xtest, ytest)

print('Quant test accuracy:', q_aware_model_accuracy)

110/110 [==============================] - 3s 25ms/step - loss: 1.2690 - accuracy: 0.8355
Quant test accuracy: 0.835464596748352


In [21]:
import tensorflow as tf
run_model = tf.function(lambda x: q_aware_model(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], q_aware_model.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/LSTM_CNN_QAT_wisdm"
q_aware_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 19:33:12.257923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:33:12.258793: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:33:12.259423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:33:12.986529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:33:12.987137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:33:12.987789: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:33:13.951428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:33:13.952257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:33:13.952869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:33:14.717149: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:33:14.717940: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:33:14.718493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT_wisdm/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT_wisdm/assets


In [22]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

2025-12-15 19:33:15.814046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:33:15.814117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:33:15.814152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:33:16.223703: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:33:16.223755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:33:16.223788: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:33:16.428527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:33:16.428580: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:33:16.428614: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [24]:
with open(f"{base_path}/models/cnn_lstm_qat_wisdm.tflite", "wb") as file:
    file.write(quantized_tflite_model)

In [23]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [25]:
interpreter = tf.lite.Interpreter(model_path=f"{base_path}/models/cnn_lstm_qat_wisdm.tflite")
interpreter.allocate_tensors()

test_accuracy_qat_tflite = evaluate_model(interpreter)
print("quant aware training model accuracy", test_accuracy_qat_tflite)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.


quant aware training model accuracy 0.8275078147200909


In [26]:
print("CNN-LSTM QAT wisdm model size in MB:", os.path.getsize(f"{base_path}/models/cnn_lstm_qat_wisdm.tflite") / float(2**20))

CNN-LSTM QAT wisdm model size in MB: 0.837127685546875
